In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

## Loading mnist dataset

In [10]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)


(60000, 28, 28)
(60000,)


In [11]:
x_train = x_train.reshape(-1, 28*28).astype("float32") / 255.0 # we divided by 255 to normalize the values to be between 0 and 1 rather than 0 and 255
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0 
# the conversion from numpy to tensor is done automatically

## Sequential api (very convenient, not very flexible)

In [31]:
# Sequential groups a linear stack of layers into a Model.
# it's not very flexible because we can only map one input to one output
model = keras.Sequential([
    keras.Input(shape=(784,)),# input layer
    layers.Dense(units=512, activation='relu'),#hidden layer
    layers.Dense(256, activation='relu', name = "my_layer"),#hidden layer
    layers.Dense(10)# output layer
])
# print(model.summary())
# model = keras.Model(inputs=model.inputs,
#                     outputs=[model.layers[-2].output
#                              ])
model = keras.Model(inputs=model.inputs,
                    outputs=[layer.output for layer in model.layers]
                             )
features = model.predict(x_train)
for feature in features:
    print(feature.shape)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step
(60000, 512)
(60000, 256)
(60000, 10)


In [21]:
# model.compile specify the network config
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [22]:
# specify the concrete training of the network
model.fit(x_train, y_train,batch_size=32,epochs=5,verbose = 2)
model.evaluate(x_test, y_test,batch_size=32,verbose = 2)

Epoch 1/5
1875/1875 - 17s - 9ms/step - accuracy: 0.9444 - loss: 0.1841
Epoch 2/5
1875/1875 - 17s - 9ms/step - accuracy: 0.9758 - loss: 0.0780
Epoch 3/5
1875/1875 - 19s - 10ms/step - accuracy: 0.9824 - loss: 0.0550
Epoch 4/5
1875/1875 - 14s - 7ms/step - accuracy: 0.9864 - loss: 0.0427
Epoch 5/5
1875/1875 - 15s - 8ms/step - accuracy: 0.9892 - loss: 0.0339
313/313 - 1s - 3ms/step - accuracy: 0.9811 - loss: 0.0712


[0.07118415832519531, 0.9811000227928162]

## Functional API(a bit more flexible)

In [24]:
inputs = keras.Input(shape=(784,))
x = layers.Dense(512,activation='relu')(inputs)
x = layers.Dense(256,activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [25]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [26]:
# specify the concrete training of the network
model.fit(x_train, y_train,batch_size=32,epochs=5,verbose = 2)
model.evaluate(x_test, y_test,batch_size=32,verbose = 2)

Epoch 1/5
1875/1875 - 26s - 14ms/step - accuracy: 0.9436 - loss: 0.1860
Epoch 2/5
1875/1875 - 23s - 12ms/step - accuracy: 0.9752 - loss: 0.0801
Epoch 3/5
1875/1875 - 24s - 13ms/step - accuracy: 0.9828 - loss: 0.0544
Epoch 4/5
1875/1875 - 24s - 13ms/step - accuracy: 0.9867 - loss: 0.0396
Epoch 5/5
1875/1875 - 22s - 12ms/step - accuracy: 0.9897 - loss: 0.0332
313/313 - 1s - 4ms/step - accuracy: 0.9758 - loss: 0.0821


[0.08205906301736832, 0.9757999777793884]

Documentation : https://keras.io/api/models/sequential/